# 17: MTC Expanded MNL Mode Choice

In [1]:
# TEST
import larix as lx
import pandas as pd
pd.set_option("display.max_columns", 999)
pd.set_option('expand_frame_repr', False)
pd.set_option('display.precision', 3)

/Users/jeffnewman/LocalGit/larix/larix/model/numbamodel.py:24: UserWarning: 

##### larix is experimental, and not feature-complete  #####
the first time you import on a new system, this package will
compile optimized binaries for your machine, which may take 
a little while, please be patient 

  warnings.warn( ### EXPERIMENTAL ### )
OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
#import sharrow.logging

In [3]:
#sharrow.logging.log_to_console(25)

For this example, we're going to re-create model 17 from the
[Self Instructing Manual](http://www.caee.utexas.edu/prof/Bhat/COURSES/LM_Draft_060131Final-060630.pdf). (pp. 128)

In [4]:
import larix as lx
d = lx.examples.MTC()
m = lx.Model(d)

We will use the usual choice and availability variables.

In [5]:
m.availability_var = 'avail'
m.choice_ca_var = 'chose'

In [6]:
from larix import P, X

m.utility_ca = (
    + X("totcost/hhinc") * P("costbyincome")
    + X("tottime * (altid <= 4)") * P("motorized_time")
    + X("tottime * (altid >= 5)") * P("nonmotorized_time")
    + X("ovtt/dist * (altid <= 4)") * P("motorized_ovtbydist")
)

The "totcost/hhinc" data is computed once as a new variable when loading the model data.
The same applies for tottime filtered by motorized modes (we harness the convenient fact
that all the motorized modes have identifying numbers 4 or less), and "ovtt/dist".

In [7]:
for a in [4,5,6]:
    m.utility_co[a] += X("hhinc") * P("hhinc#{}".format(a))

Since the model we want to create groups together DA, SR2 and SR3+ jointly as
reference alternatives with respect to income, we can simply omit all of these alternatives
from the block that applies to **hhinc**.

For vehicles per worker, the preferred model include a joint parameter on SR2 and SR3+,
but not including DA and not fixed at zero.  Here we might use a shadow_parameter (also
called an alias in some places), which allows
us to specify one or more parameters that are simply a fixed proportion of another parameter.
For example, we can say that vehbywrk_SR2 will be equal to vehbywrk_SR.

In [8]:
for i in d['altnames'][1:3]:
    name = str(i.values)
    a = int(i.altid)
    m.utility_co[a] += (
        + X("vehbywrk") * P("vehbywrk_SR")
        + X("wkccbd+wknccbd") * P("wkcbd_"+name)
        + X("wkempden") * P("wkempden_"+name)
        + P("ASC_"+name)
    )

for i in d['altnames'][3:]:
    name = str(i.values)
    a = int(i.altid)
    m.utility_co[a] += (
        + X("vehbywrk") * P("vehbywrk_"+name)
        + X("wkccbd+wknccbd") * P("wkcbd_"+name)
        + X("wkempden") * P("wkempden_"+name)
        + P("ASC_"+name)
    )

We didn't explicitly define our parameters first, which is fine; Larch will
find them in the utility functions (or elsewhere in more complex models).
But they may be found in a weird order that is hard to read in reports.
We can define an ordering scheme by assigning to the parameter_groups attribute,
like this:

In [9]:
m.ordering = (
    ('LOS', ".*cost.*", ".*time.*", ".*dist.*",),
    ('Zonal', "wkcbd.*", "wkempden.*",),
    ('Household', "hhinc.*", "vehbywrk.*",),
    ('ASCs', "ASC.*",),
)
m.set_cap(25)

Each item in parameter_ordering is a tuple, with a label and one or more regular expressions,
which will be compared against
all the parameter names.  Any names that match will be pulled out and put into the
reporting order sequentially.  Thus if a parameter name would match more than one
regex, it will appear in the ordering only for the first match.


Having created this model, we can then estimate it:

In [10]:
result = m.maximize_loglike(stderr=True)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Larch Model Dashboard                                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ optimization complete                                                                     │
│ Log Likelihood Current =      -3444.185059 Best =      -3444.185059                       │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┓ │
│ ┃                               ┃                ┃               ┃            ┃ Null    ┃ │
│ ┃ Parameter                     ┃ Estimate       ┃ Std. Error    ┃ t-Stat     ┃ Val     ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━┩ │
│ │ ASC_Bike                      │ -1.6287138     │  0.4274419    │ -3.81      │  0      │ │
│ │ ASC_SR2                       │ -1.8076754     │  0.10612901   │ -17.03     │  0      │ │
│ │ ASC_SR3+                      │ -3.4337739     │  0.15187916   │ -22.61     │  0      │ │
│ │ ASC_Transit                   │ -0.68518247    │  0.24779683   │ -2.765     │  0      │ │
│ │ ASC_Walk                      │  0.069445899   │  0.34801513   │  0.1995    │  0      │ │
│ │ costbyincome                  │ -0.052404614   │  0.010404362  │ -5.037     │  0      │ │
│ │ hhinc#4                       │ -0.0053241481  │  0.001977009  │ -2.693     │  0      │ │
│ │ hhinc#5                       │ -0.0086347327  │  0.0051542935 │ -1.675     │  0      │ │
│ │ hhinc#6                       │ -0.0059967426  │  0.0031488179 │ -1.904     │  0      │ │
│ │ motorized_ovtbydist           │ -0.13269663    │  0.019634474  │ -6.758     │  0      │ │
│ │ motorized_time                │ -0.020198723   │  0.0038145634 │ -5.295     │  0      │ │
│ │ nonmotorized_time             │ -0.045460512   │  0.0057690404 │ -7.88      │  0      │ │
│ │ vehbywrk_Bike                 │ -0.70273389    │  0.25837421   │ -2.72      │  0      │ │
│ │ vehbywrk_SR                   │ -0.31676515    │  0.066640146  │ -4.753     │  0      │ │
│ │ vehbywrk_Transit              │ -0.94611812    │  0.11828256   │ -7.999     │  0      │ │
│ │ vehbywrk_Walk                 │ -0.72238378    │  0.16942607   │ -4.264     │  0      │ │
│ │ wkcbd_Bike                    │  0.48876356    │  0.36118519   │  1.353     │  0      │ │
│ │ wkcbd_SR2                     │  0.25973914    │  0.12335605   │  2.106     │  0      │ │
│ │ wkcbd_SR3+                    │  1.0693081     │  0.19129156   │  5.59      │  0      │ │
│ │ wkcbd_Transit                 │  1.3089882     │  0.16568337   │  7.901     │  0      │ │
│ │ wkcbd_Walk                    │  0.10212882    │  0.25211287   │  0.4051    │  0      │ │
│ │ wkempden_Bike                 │  0.0019360529  │  0.0012142073 │  1.594     │  0      │ │
│ │ wkempden_SR2                  │  0.0015795061  │  0.000390367… │  4.046     │  0      │ │
│ │ wkempden_SR3+                 │  0.0022587619  │  0.000451989… │  4.997     │  0      │ │
│ │ wkempden_Transit              │  0.0031341311  │  0.000360776… │  8.687     │  0      │ │
│ │ wkempden_Walk                 │  0.0028903625  │  0.0007422276 │  3.894     │  0      │ │
│ └───────────────────────────────┴────────────────┴───────────────┴────────────┴─────────┘ │
└───────────────────────────────────────────────────────────────────────────────────────────┘

In [11]:
# TEST
r = result
from pytest import approx
assert r.loglike == approx(-3444.185105027836)
assert r.n_cases == 5029
assert 'success' in r.message.lower()
assert dict(zip(m.pnames, r.x)) == approx({
    'ASC_Bike': -1.6287137598928783,
    'ASC_SR2': -1.8076753640695755,
    'ASC_SR3+': -3.4337739234532756,
    'ASC_Transit': -0.6851824718405783,
    'ASC_Walk': 0.06944589897084852,
    'costbyincome': -0.05240461370499937,
    'hhinc#4': -0.0053241481308512004,
    'hhinc#5': -0.008634732662554195,
    'hhinc#6': -0.005996742608087729,
    'motorized_ovtbydist': -0.1326966328086029,
    'motorized_time': -0.02019872321825129,
    'nonmotorized_time': -0.04546051181623673,
    'vehbywrk_Bike': -0.7027338887881739,
    'vehbywrk_SR': -0.3167651458729014,
    'vehbywrk_Transit': -0.946118123994283,
    'vehbywrk_Walk': -0.7223837768020874,
    'wkcbd_Bike': 0.4887635620321093,
    'wkcbd_SR2': 0.25973913680020616,
    'wkcbd_SR3+': 1.0693080706303768,
    'wkcbd_Transit': 1.3089881907595406,
    'wkcbd_Walk': 0.10212881871476903,
    'wkempden_Bike': 0.001936052921612131,
    'wkempden_SR2': 0.001579506064082312,
    'wkempden_SR3+': 0.002258761866013131,
    'wkempden_Transit': 0.0031341311451905467,
    'wkempden_Walk': 0.002890362519991945,
})

In [12]:
m.parameter_summary()

In [13]:
# TEST
from larix.util.testing import assert_same_text
assert_same_text(
    m.parameter_summary().data.to_markdown(),
    """
    |                                   |    Value |   Std Err |   t Stat | Signif   |   Null Value |
    |:----------------------------------|---------:|----------:|---------:|:---------|-------------:|
    | ('LOS', 'costbyincome')           | -0.0524  |  0.0104   |    -5.04 | ***      |            0 |
    | ('LOS', 'motorized_time')         | -0.0202  |  0.00381  |    -5.3  | ***      |            0 |
    | ('LOS', 'nonmotorized_time')      | -0.0455  |  0.00577  |    -7.88 | ***      |            0 |
    | ('LOS', 'motorized_ovtbydist')    | -0.133   |  0.0196   |    -6.76 | ***      |            0 |
    | ('Zonal', 'wkcbd_Bike')           |  0.489   |  0.361    |     1.35 |          |            0 |
    | ('Zonal', 'wkcbd_SR2')            |  0.26    |  0.123    |     2.11 | *        |            0 |
    | ('Zonal', 'wkcbd_SR3+')           |  1.07    |  0.191    |     5.59 | ***      |            0 |
    | ('Zonal', 'wkcbd_Transit')        |  1.31    |  0.166    |     7.9  | ***      |            0 |
    | ('Zonal', 'wkcbd_Walk')           |  0.102   |  0.252    |     0.41 |          |            0 |
    | ('Zonal', 'wkempden_Bike')        |  0.00194 |  0.00121  |     1.59 |          |            0 |
    | ('Zonal', 'wkempden_SR2')         |  0.00158 |  0.00039  |     4.05 | ***      |            0 |
    | ('Zonal', 'wkempden_SR3+')        |  0.00226 |  0.000452 |     5    | ***      |            0 |
    | ('Zonal', 'wkempden_Transit')     |  0.00313 |  0.000361 |     8.69 | ***      |            0 |
    | ('Zonal', 'wkempden_Walk')        |  0.00289 |  0.000742 |     3.89 | ***      |            0 |
    | ('Household', 'hhinc#4')          | -0.00532 |  0.00198  |    -2.69 | **       |            0 |
    | ('Household', 'hhinc#5')          | -0.00863 |  0.00515  |    -1.68 |          |            0 |
    | ('Household', 'hhinc#6')          | -0.006   |  0.00315  |    -1.9  |          |            0 |
    | ('Household', 'vehbywrk_Bike')    | -0.703   |  0.258    |    -2.72 | **       |            0 |
    | ('Household', 'vehbywrk_SR')      | -0.317   |  0.0666   |    -4.75 | ***      |            0 |
    | ('Household', 'vehbywrk_Transit') | -0.946   |  0.118    |    -8    | ***      |            0 |
    | ('Household', 'vehbywrk_Walk')    | -0.722   |  0.169    |    -4.26 | ***      |            0 |
    | ('ASCs', 'ASC_Bike')              | -1.63    |  0.427    |    -3.81 | ***      |            0 |
    | ('ASCs', 'ASC_SR2')               | -1.81    |  0.106    |   -17.03 | ***      |            0 |
    | ('ASCs', 'ASC_SR3+')              | -3.43    |  0.152    |   -22.61 | ***      |            0 |
    | ('ASCs', 'ASC_Transit')           | -0.685   |  0.248    |    -2.77 | **       |            0 |
    | ('ASCs', 'ASC_Walk')              |  0.0694  |  0.348    |     0.2  |          |            0 |
    """
)